In [ ]:
ref_osm = '/mnt/d/osm2lulc01/osm2lulc01.gdb/osm/points_evu_forest'

cols_keys = [
    #'aeroway', 'amenity', 'barrier', 'building',
    #'craft', 'geological', 'historic', 'landuse', 'leisure',
    #'man_made', 'military', 'natural', 'office', 'place', 'shop',
    #'sport', 'tourism', 'waterway', 'power', 'railway', 'healthcare',
    #'highway', 'water', 'landcover', 'industrial', 'leaf_type',
    #'material', 'surface', 'leaf_cycle',
    #
    #'highway', 'waterway', 'aerialway',
    #'barrier', 'man_made', 'railway', 'sidewalk', 'busway', 'natural',
    #'leisure', 'cycleway'
    #
    'barrier', 'highway',
    'place', 'man_made', 'building', 'natural',
    'leisure', 'healthcare', 'railway', 'amenity', 'waterway',
    'landuse', 'power', 'shop',
    'tourism', 'historic', 'cuisine', 'sport', 'office', 'religion'
]

col_class = 'code_2018'

out_tbl = '/mnt/d/osm2lulc01/tags_statistics3.xlsx'

In [ ]:
import pandas as pd

from glass.rd.shp import shp_to_obj
from glass.dtr.mge.pd import merge_df
from glass.wt import obj_to_tbl
from glass.prop.gpd import geom_type

In [ ]:
gattr = 'gattr'

In [ ]:
gdf = shp_to_obj(ref_osm)

gtype = geom_type(gdf)

classes = gdf[col_class].unique()

if gtype == 'MultiPolygon' or gtype == 'Polygon':
    gdf[gattr] = gdf.geometry.area

elif gtype == 'MultiLineString' or gtype == 'LineString':
    gdf[gattr] = gdf.geometry.length

In [ ]:
gtype

In [ ]:
gdf.columns.values

In [ ]:
if gtype == 'MultiPoint' or gtype == 'Point':
    class_df = pd.DataFrame({
        gattr : gdf.groupby([col_class])[col_class].agg('count')
    }).reset_index()

else:
    class_df = pd.DataFrame({
        gattr : gdf.groupby([col_class])[gattr].agg('sum')
    }).reset_index()

total_area = class_df[gattr].sum()

In [ ]:
cls_area = {}

for i, r in class_df.iterrows():
    cls_area[r[col_class]] = r[gattr]

In [ ]:
cls_area

In [ ]:
class_df

In [ ]:
# For each key, list values

key_df = {}
for cls in classes:
    cls_df = gdf[gdf[col_class] == cls]

    dfs = []
    
    for _key in cols_keys:
        _df = cls_df[~cls_df[_key].isnull()]

        if gtype == 'MultiPoint' or gtype == 'Point':
            gb = pd.DataFrame({
                gattr : _df.groupby([_key])[_key].agg('count')
            }).reset_index()

        else:
            gb = pd.DataFrame({
                gattr : _df.groupby([_key])[gattr].agg('sum')
            }).reset_index()

        gb['osmtag'] = _key + '=' + gb[_key]

        gb.drop(_key, axis=1, inplace=True)

        gb[f'p_{cls}'] = gb[gattr] / cls_area[cls] * 100

        gb[f'pt_{cls}'] = gb[gattr] / total_area * 100

        dfs.append(gb)

    key_df[cls] = merge_df(dfs)

In [ ]:
# Join all dataframes of each class

main_df = key_df[classes[0]]

for cls in classes[1:]:
    key_df[cls].rename(columns={'osmtag' : 'osmtag_y'}, inplace=True)

    main_df = main_df.merge(key_df[cls], how='outer', left_on='osmtag', right_on='osmtag_y')

    main_df['osmtag'] = main_df.osmtag.fillna(value=main_df.osmtag_y)

    main_df.drop('osmtag_y', axis=1, inplace=True)

    main_df = main_df.fillna(0)

In [ ]:
obj_to_tbl(main_df, out_tbl, sheetsName='statistics')